In [1]:
from ultralytics import YOLO
import json
import glob
import os
import cv2
import numpy as np
import random
import matplotlib.pyplot as plt

# Memuat model YOLO
model = YOLO("D:\\DATA RAFKI\\PunyaResti\\PunyaResti\\DATASET SMARTPHONE\\4CH\\best.pt")  # Ganti dengan path model Anda
class_names = model.names  # Dapatkan daftar label kelas

# Path folder gambar
image_folder = "D:\\DATA RAFKI\\PunyaResti\\PunyaResti\\DATASET SMARTPHONE\\4CH\\test\\images"  # Ganti dengan folder gambar Anda
image_paths = glob.glob(os.path.join(image_folder, "*.jpg"))

# Pastikan folder output untuk JSON dan hasil visualisasi ada
json_output_folder = "D:\\DATA RAFKI\\PunyaResti\\PunyaResti\\DATASET SMARTPHONE\\4CH\\TEST_4CH_JSON"
image_output_folder = "D:\\DATA RAFKI\\PunyaResti\\PunyaResti\\DATASET SMARTPHONE\\4CH\\TEST_4CH_IMG"
os.makedirs(json_output_folder, exist_ok=True)
os.makedirs(image_output_folder, exist_ok=True)

# Looping melalui setiap gambar dalam folder
for image_path in image_paths:
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    overlay = image.copy()

    results = model(image_path)

    image_results = {
        "version": "5.5.0",
        "flags": {},
        "shapes": [],
        "imagePath": os.path.basename(image_path)
    }

    confidence_texts = []
    for result in results:
        if result.masks:
            for mask, box in zip(result.masks.xy, result.boxes):
                confidence = box.conf.item()
                class_id = int(box.cls.item())
                class_label = class_names[class_id]
                polygon_points = mask.tolist()

                image_results["shapes"].append({
                    "label": class_label,
                    "points": polygon_points,
                    "group_id": None,
                    "description": "",
                    "shape_type": "polygon",
                    "flags": {},
                    "mask": None
                })

                # Konversi mask ke format integer untuk OpenCV
                polygon = np.array(mask, dtype=np.int32).reshape((-1, 1, 2))

                # Pilih warna acak untuk visualisasi
                color = [random.randint(0, 255) for _ in range(3)]

                # Gambar poligon di gambar asli
                cv2.polylines(image, [polygon], isClosed=True, color=color, thickness=2)
                cv2.fillPoly(overlay, [polygon], color=color)

                # Menentukan titik tengah untuk menampilkan confidence
                centroid = np.mean(polygon, axis=0).astype(int).flatten()
                cv2.putText(image, f"{class_label}: {confidence:.2f}", tuple(centroid),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)

                # Simpan confidence text untuk ditampilkan di sudut kanan atas
                confidence_texts.append(f"{class_label}: {confidence:.2f}")

    # Tambahkan confidence summary di sudut kanan atas
    y_offset = 50
    for text in confidence_texts:
        cv2.putText(image, text, (image.shape[1] - 300, y_offset), cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 5, cv2.LINE_AA)
        y_offset += 60

    # Menggabungkan overlay dengan transparansi
    alpha = 0.4
    image = cv2.addWeighted(overlay, alpha, image, 1 - alpha, 0)

    # Simpan hasil dalam format JSON
    json_filename = os.path.splitext(os.path.basename(image_path))[0] + ".json"
    json_path = os.path.join(json_output_folder, json_filename)
    with open(json_path, "w") as f:
        json.dump(image_results, f, indent=4)

    # Simpan gambar hasil visualisasi
    output_image_path = os.path.join(image_output_folder, os.path.basename(image_path))
    plt.figure(figsize=(8, 6))
    plt.imshow(image)
    plt.axis("off")
    plt.title(f"Segmentasi: {os.path.basename(image_path)}")
    plt.savefig(output_image_path, bbox_inches="tight")
    plt.close()

    print(f"Prediksi untuk {image_path} disimpan di {json_path} dan hasil visualisasi di {output_image_path}")


image 1/1 D:\DATA RAFKI\PunyaResti\PunyaResti\DATASET SMARTPHONE\4CH\test\images\ABBAS-MURTHADO_NORMAL_4CH_frame_124.jpg: 384x640 1 IVS, 1 IAS, 398.3ms
Speed: 3.2ms preprocess, 398.3ms inference, 2.3ms postprocess per image at shape (1, 3, 384, 640)
Prediksi untuk D:\DATA RAFKI\PunyaResti\PunyaResti\DATASET SMARTPHONE\4CH\test\images\ABBAS-MURTHADO_NORMAL_4CH_frame_124.jpg disimpan di D:\DATA RAFKI\PunyaResti\PunyaResti\DATASET SMARTPHONE\4CH\TEST_4CH_JSON\ABBAS-MURTHADO_NORMAL_4CH_frame_124.json dan hasil visualisasi di D:\DATA RAFKI\PunyaResti\PunyaResti\DATASET SMARTPHONE\4CH\TEST_4CH_IMG\ABBAS-MURTHADO_NORMAL_4CH_frame_124.jpg

image 1/1 D:\DATA RAFKI\PunyaResti\PunyaResti\DATASET SMARTPHONE\4CH\test\images\ABBAS-MURTHADO_NORMAL_4CH_frame_161.jpg: 384x640 1 IVS, 1 IAS, 366.9ms
Speed: 1.9ms preprocess, 366.9ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)
Prediksi untuk D:\DATA RAFKI\PunyaResti\PunyaResti\DATASET SMARTPHONE\4CH\test\images\ABBAS-MURTHADO_NORMAL_4